In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime 
import swifter

from sklearn.ensemble import RandomForestRegressor

In [3]:
# データ読み込み
train = pd.read_csv('train.csv')
fact = pd.read_csv('test.csv')

print(train.shape,fact.shape)

(2544196, 9) (636049, 7)


In [4]:
# データ確認
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544196 entries, 0 to 2544195
Data columns (total 9 columns):
id                object
pick_datetime     object
pick_dayofweek    int64
pick_latitude     float64
pick_longitude    float64
drop_datetime     object
drop_dayofweek    int64
drop_latitude     float64
drop_longitude    float64
dtypes: float64(4), int64(2), object(3)
memory usage: 174.7+ MB


In [5]:
fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636049 entries, 0 to 636048
Data columns (total 7 columns):
id                636049 non-null object
pick_datetime     636049 non-null object
pick_dayofweek    636049 non-null int64
pick_latitude     636049 non-null float64
pick_longitude    636049 non-null float64
drop_latitude     636049 non-null float64
drop_longitude    636049 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 34.0+ MB


In [6]:
# データ前処理

In [7]:
#＃　locationの外れ値処理
train = train[train['pick_latitude'] >30]
train = train[train['drop_latitude'] >30]

print(train.shape,fact.shape)

(2544194, 9) (636049, 7)


In [8]:
## 文字列型->datetime型へ変換
train['pick_datetime'] = pd.to_datetime(train['pick_datetime'])
train['drop_datetime'] = pd.to_datetime(train['drop_datetime'])
fact['pick_datetime'] = pd.to_datetime(fact['pick_datetime'])


In [9]:
##　目的変数の設定

def delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()

### trip_duraction（単位:秒）を追加
train['trip_duraction'] = train[['pick_datetime', 'drop_datetime']].swifter.apply(delta, axis=1)

### drop_datetime , drop_dayofweek 削除
train = train.drop('drop_datetime', axis=1)
train = train.drop('drop_dayofweek', axis=1)

### 目的変数と説明変数の分離
target = train['trip_duraction']
train = train.drop('trip_duraction', axis=1)

In [10]:
train.head(3)

,id,pick_datetime,pick_dayofweek,pick_latitude,pick_longitude,drop_latitude,drop_longitude
0,id_0000661020,2260-01-07 11:11:45,7,35.708821,139.754566,35.706122,139.768515
1,id_0002302407,2260-01-12 12:33:32,5,35.659465,139.722541,35.661145,139.730093
2,id_0000978303,2260-01-26 14:12:08,5,35.688164,139.771702,35.708068,139.732200


In [11]:
## pick_datetime
### 日付から年、月、日をそれぞれ分離
# train['year'] = train['pick_datetime'].dt.year
# train['month'] = train['pick_datetime'].dt.month
# train['day'] = train['pick_datetime'].dt.day
train['hour'] = train['pick_datetime'].dt.hour
# train['minute'] = train['pick_datetime'].dt.minute
# train['second'] = train['pick_datetime'].dt.second

# fact['year'] = fact['pick_datetime'].dt.year
# fact['month'] = fact['pick_datetime'].dt.month
# fact['day'] = fact['pick_datetime'].dt.day
fact['hour'] = fact['pick_datetime'].dt.hour
# fact['minute'] = fact['pick_datetime'].dt.minute
# fact['second'] = fact['pick_datetime'].dt.second

### 元のpick_datetime削除
train = train.drop('pick_datetime', axis=1)
fact = fact.drop('pick_datetime', axis=1)


In [12]:
# hour　を　morning(5-11),afternoon(11-16),evening(16-18),night(18-24),deepnight(24-5)に分割




In [13]:
# dayofweek を weekdays(平日)とweekends(土日)に分割
train['pick_dayofweek'] = train['pick_dayofweek'].replace({1: 'sunday' , 2:'weekdays', 3:'weekdays', 4:'weekdays', 5:'weekdays', 6:'friday', 7:'saturday'})

fact['pick_dayofweek'] = fact['pick_dayofweek'].replace({1: 'sunday' , 2:'weekdays', 3:'weekdays', 4:'weekdays', 5:'weekdays', 6:'friday', 7:'saturday'})


In [14]:
# type(train['day'][0])

In [15]:
# dayを月末と月初に変換
# train['day'].where( train['day'] <= 5 , '月初' , inplace=True)
# train['day'].where( 5 < train['day'] < 25 , '中旬' , inplace=True)
# train['day'].where( train['day'] >= 25 , '月末' , inplace=True)



# train.loc[train['day'] < 5] = '月初'
# train.loc[train['day'] >= 25] = '月末'
# train.loc[5 < train['day'] < 25] = '中旬'

# fact.loc[train['day'] <= 5] = '月初'
# fact.loc[train['day'] >= 25] = '月末'
# fact.loc[5 < train['day'] < 25] = '中旬'


In [16]:
## idの削除
train = train.drop('id', axis=1)

submit = pd.DataFrame()
submit['id'] = fact['id']
fact = fact.drop('id', axis=1)


In [17]:
train.head(3)

,pick_dayofweek,pick_latitude,pick_longitude,drop_latitude,drop_longitude,hour
0,saturday,35.708821,139.754566,35.706122,139.768515,11
1,weekdays,35.659465,139.722541,35.661145,139.730093,12
2,weekdays,35.688164,139.771702,35.708068,139.732200,14


In [18]:
fact.head(3)

,pick_dayofweek,pick_latitude,pick_longitude,drop_latitude,drop_longitude,hour
0,weekdays,35.654196,139.722224,35.682485,139.763255,23
1,weekdays,35.659671,139.695681,35.682289,139.682441,12
2,friday,35.658575,139.697312,35.661905,139.729502,13


In [19]:
target.head(3)

0    316.0
1    366.0
2    949.0
Name: trip_duraction, dtype: float64

In [20]:
## pick_dayofweekのダミー変数化
all_explainData = pd.concat([train, fact], axis=0)
dummy_col = ['pick_dayofweek']
all_explainData = pd.get_dummies(all_explainData, columns = dummy_col, drop_first=False)


In [21]:
all_explainData.head(3)

,pick_latitude,pick_longitude,drop_latitude,drop_longitude,hour,pick_dayofweek_friday,pick_dayofweek_saturday,pick_dayofweek_sunday,pick_dayofweek_weekdays
0,35.708821,139.754566,35.706122,139.768515,11,0,1,0,0
1,35.659465,139.722541,35.661145,139.730093,12,0,0,0,1
2,35.688164,139.771702,35.708068,139.732200,14,0,0,0,1


In [22]:
### 元のサイズにそれぞれ戻す
train = all_explainData[:len(train)]
fact = all_explainData[len(train):]

In [23]:
train.head(3)

,pick_latitude,pick_longitude,drop_latitude,drop_longitude,hour,pick_dayofweek_friday,pick_dayofweek_saturday,pick_dayofweek_sunday,pick_dayofweek_weekdays
0,35.708821,139.754566,35.706122,139.768515,11,0,1,0,0
1,35.659465,139.722541,35.661145,139.730093,12,0,0,0,1
2,35.688164,139.771702,35.708068,139.732200,14,0,0,0,1


In [24]:
fact.head(3)

,pick_latitude,pick_longitude,drop_latitude,drop_longitude,hour,pick_dayofweek_friday,pick_dayofweek_saturday,pick_dayofweek_sunday,pick_dayofweek_weekdays
0,35.654196,139.722224,35.682485,139.763255,23,0,0,0,1
1,35.659671,139.695681,35.682289,139.682441,12,0,0,0,1
2,35.658575,139.697312,35.661905,139.729502,13,1,0,0,0


In [25]:
# 学習
DT = RandomForestRegressor(random_state=666)
DT.fit(train,target)


/Users/onikinagisa/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=666, verbose=0,
                      warm_start=False)

In [26]:
## 本番予測
pred = DT.predict(fact)

print(pred)

[1190.7  743.7  894.4 ... 1369.5  657.8 1032.3]


In [27]:
submit['pred'] = pred

In [28]:
submit.to_csv('result.csv' ,header=False, index=False)